TEST 

In [2]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import torch 
import random 

#pytoch imports

import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


In [4]:
b1 =[torch.LongTensor([i for i in range(5*j,5*j+5-random.randint(0,2))]) for j in range(1,4)]
#b2 =[torch.LongTensor([i for i in range(5*j,5*j+5)]) for j in range(1,4)]

b1_lenghts = [len(ex) for ex in b1]
    

b1

b1_lenghts

[tensor([5, 6, 7]), tensor([10, 11, 12]), tensor([15, 16, 17, 18])]

[3, 3, 4]

In [5]:
max_tokens = 7
b1[0] = nn.ConstantPad1d((0,max_tokens-b1[0].shape[0]),0)(b1[0])

b1

[tensor([5, 6, 7, 0, 0, 0, 0]), tensor([10, 11, 12]), tensor([15, 16, 17, 18])]

In [6]:
padded_b1 = rnn.pad_sequence(b1,batch_first=True,padding_value=0)

padded_b1
padded_b1.shape

tensor([[ 5,  6,  7,  0,  0,  0,  0],
        [10, 11, 12,  0,  0,  0,  0],
        [15, 16, 17, 18,  0,  0,  0]])

torch.Size([3, 7])

In [7]:
fake_matrix = [[random.randint(1,7) for _ in range(4)] for _ in range(19)]
fake_matrix.insert(0,[0,0,0,0])
fake_matrix 

[[0, 0, 0, 0],
 [7, 6, 6, 7],
 [1, 1, 1, 1],
 [1, 7, 5, 6],
 [7, 2, 6, 3],
 [3, 2, 5, 3],
 [2, 2, 6, 2],
 [3, 4, 2, 7],
 [1, 2, 1, 4],
 [7, 4, 3, 3],
 [5, 5, 6, 2],
 [4, 3, 2, 4],
 [3, 2, 7, 4],
 [7, 3, 2, 5],
 [1, 6, 6, 4],
 [6, 5, 5, 3],
 [2, 6, 6, 7],
 [2, 3, 1, 2],
 [5, 4, 5, 2],
 [7, 3, 3, 2]]

In [8]:
fake_matrix = torch.Tensor(fake_matrix)   #the embedding matrix 
_ , embedding_dim = fake_matrix.shape 
emb_layer = nn.Embedding.from_pretrained(fake_matrix, freeze=False, padding_idx = 0)

In [9]:
emb = emb_layer(padded_b1)

emb.shape
emb

torch.Size([3, 7, 4])

tensor([[[3., 2., 5., 3.],
         [2., 2., 6., 2.],
         [3., 4., 2., 7.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[5., 5., 6., 2.],
         [4., 3., 2., 4.],
         [3., 2., 7., 4.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[6., 5., 5., 3.],
         [2., 6., 6., 7.],
         [2., 3., 1., 2.],
         [5., 4., 5., 2.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]], grad_fn=<EmbeddingBackward0>)

In [10]:
packed_embeds = pack_padded_sequence(emb,b1_lenghts,batch_first=True, enforce_sorted=False)

packed_embeds

PackedSequence(data=tensor([[6., 5., 5., 3.],
        [3., 2., 5., 3.],
        [5., 5., 6., 2.],
        [2., 6., 6., 7.],
        [2., 2., 6., 2.],
        [4., 3., 2., 4.],
        [2., 3., 1., 2.],
        [3., 4., 2., 7.],
        [3., 2., 7., 4.],
        [5., 4., 5., 2.]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([3, 3, 3, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))

In [11]:
re_padded_embeds = pad_packed_sequence(packed_embeds,batch_first=True)

re_padded_embeds

(tensor([[[3., 2., 5., 3.],
          [2., 2., 6., 2.],
          [3., 4., 2., 7.],
          [0., 0., 0., 0.]],
 
         [[5., 5., 6., 2.],
          [4., 3., 2., 4.],
          [3., 2., 7., 4.],
          [0., 0., 0., 0.]],
 
         [[6., 5., 5., 3.],
          [2., 6., 6., 7.],
          [2., 3., 1., 2.],
          [5., 4., 5., 2.]]], grad_fn=<IndexSelectBackward0>),
 tensor([3, 3, 4]))

In [12]:
emb_reshaped = emb.permute(0,2,1)

emb_reshaped.shape

emb_reshaped

torch.Size([3, 4, 7])

tensor([[[3., 2., 3., 0., 0., 0., 0.],
         [2., 2., 4., 0., 0., 0., 0.],
         [5., 6., 2., 0., 0., 0., 0.],
         [3., 2., 7., 0., 0., 0., 0.]],

        [[5., 4., 3., 0., 0., 0., 0.],
         [5., 3., 2., 0., 0., 0., 0.],
         [6., 2., 7., 0., 0., 0., 0.],
         [2., 4., 4., 0., 0., 0., 0.]],

        [[6., 2., 2., 5., 0., 0., 0.],
         [5., 6., 3., 4., 0., 0., 0.],
         [5., 6., 1., 5., 0., 0., 0.],
         [3., 7., 2., 2., 0., 0., 0.]]], grad_fn=<PermuteBackward0>)

In [13]:
dense_layer = nn.Linear(max_tokens,1)
sentence_emb = dense_layer(emb_reshaped)

sentence_emb

sentence_emb = sentence_emb.squeeze(2)

sentence_emb
sentence_emb.shape


tensor([[[ 1.1214e+00],
         [ 1.0051e+00],
         [ 7.0928e-01],
         [ 1.7289e+00]],

        [[ 1.2594e+00],
         [ 1.3067e+00],
         [ 2.5334e+00],
         [ 6.0683e-01]],

        [[ 1.2158e+00],
         [ 4.1463e-01],
         [-7.7271e-04],
         [-2.4944e-01]]], grad_fn=<AddBackward0>)

tensor([[ 1.1214e+00,  1.0051e+00,  7.0928e-01,  1.7289e+00],
        [ 1.2594e+00,  1.3067e+00,  2.5334e+00,  6.0683e-01],
        [ 1.2158e+00,  4.1463e-01, -7.7271e-04, -2.4944e-01]],
       grad_fn=<SqueezeBackward1>)

torch.Size([3, 4])

In [14]:
class model(nn.Module):
    """
        Class defining our model architecture  
    """
    def __init__(self, emb_matrix , pad_idx: int, max_tokens: int) :
        super().__init__()

        self.embedding_layer, self.word_embedding_dim = self.create_emb_layer(emb_matrix,pad_idx)

        self.pad_idx = pad_idx

        self.max_tokens = max_tokens

        self.rnn = nn.LSTM(self.word_embedding_dim, self.word_embedding_dim,bidirectional = False,batch_first = True) 

    def create_emb_layer(self,weights_matrix, pad_idx : int):
        
        matrix = torch.Tensor(weights_matrix)   #the embedding matrix 
        _ , embedding_dim = matrix.shape 
        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=True, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable 
        
        return emb_layer, embedding_dim


    def forward(self, claims, claim_lengths):

        claims = claims.copy()
        
        claims[0] = nn.ConstantPad1d((0,max_tokens-claims[0].shape[0]),0)(claims[0])

        padded_claims = rnn.pad_sequence(claims,batch_first = True, padding_value = self.pad_idx)         

        embed_claims = self.embedding_layer(padded_claims)

        packed_embeds = pack_padded_sequence(embed_claims, claim_lengths, batch_first=True, enforce_sorted=False)   #pack the sentences batch so that no unnecessary computation is performed for padding tokens
        
        packed_out, (hn,_)  = self.rnn(packed_embeds)

        unpacked_out, l = pad_packed_sequence(packed_out,batch_first=True)

        return unpacked_out,l 
    
        

In [15]:
a =[torch.LongTensor([i for i in range(5*j,5*j+5-random.randint(0,2))]) for j in range(1,4)]
a_lenths = [len(ex) for ex in a]

In [16]:
a

[tensor([5, 6, 7, 8, 9]),
 tensor([10, 11, 12, 13, 14]),
 tensor([15, 16, 17, 18, 19])]

In [17]:
m = model(fake_matrix,0,max_tokens)

p,l = m(a,a_lenths)

p.shape

p


a1 = p.sum(dim=1).div(p.count_nonzero(dim=1))
a2 = torch.sum(p,dim=1).div(1.7)

a1.shape

a2.shape

torch.Size([3, 5, 4])

tensor([[[-2.3150e-03,  1.1964e-01,  1.8562e-02,  1.1395e-01],
         [-9.0859e-03,  1.6826e-01,  1.5436e-02,  1.0533e-01],
         [-5.2416e-04,  1.0400e-01,  6.5782e-02,  1.5094e-01],
         [-9.7087e-03,  1.4114e-01,  1.9908e-01,  2.1488e-01],
         [-1.1714e-03,  5.5641e-02,  1.3532e-01,  4.6897e-01]],

        [[-1.1281e-03,  7.7209e-02,  2.0896e-03,  2.9812e-01],
         [-3.7012e-03,  8.2916e-02,  3.3154e-02,  3.2189e-01],
         [-7.8853e-03,  1.3093e-01,  1.4552e-02,  8.5223e-02],
         [-9.6754e-04,  5.0806e-02,  2.4391e-02,  3.0044e-01],
         [-7.3871e-03,  6.7545e-02,  1.1624e-02,  2.6618e-01]],

        [[-5.2273e-04,  5.5297e-02,  2.4510e-03,  3.3444e-01],
         [-3.8958e-04,  4.1549e-02,  7.7541e-03,  1.2719e-01],
         [-3.8990e-02,  1.0316e-01,  7.9965e-02,  3.6874e-01],
         [-7.1470e-03,  9.4732e-02,  3.4568e-02,  3.9351e-01],
         [-5.1282e-03,  7.7252e-02,  2.3766e-02,  4.8991e-01]]],
       grad_fn=<IndexSelectBackward0>)

torch.Size([3, 4])

torch.Size([3, 4])

In [18]:
a1

a2

tensor([[-0.0046,  0.1177,  0.0868,  0.2108],
        [-0.0042,  0.0819,  0.0172,  0.2544],
        [-0.0104,  0.0744,  0.0297,  0.3428]], grad_fn=<DivBackward0>)

tensor([[-0.0134,  0.3463,  0.2554,  0.6200],
        [-0.0124,  0.2408,  0.0505,  0.7481],
        [-0.0307,  0.2188,  0.0874,  1.0081]], grad_fn=<DivBackward0>)

In [19]:
m1 = torch.cat((a1,a2),dim=1)
m1

tensor([[-0.0046,  0.1177,  0.0868,  0.2108, -0.0134,  0.3463,  0.2554,  0.6200],
        [-0.0042,  0.0819,  0.0172,  0.2544, -0.0124,  0.2408,  0.0505,  0.7481],
        [-0.0104,  0.0744,  0.0297,  0.3428, -0.0307,  0.2188,  0.0874,  1.0081]],
       grad_fn=<CatBackward0>)

In [20]:
m2 = torch.stack((a1,a2), dim=0).sum(dim=0)
m2

tensor([[-0.0180,  0.4640,  0.3422,  0.8309],
        [-0.0166,  0.3227,  0.0676,  1.0025],
        [-0.0411,  0.2932,  0.1171,  1.3509]], grad_fn=<SumBackward1>)

In [21]:
m3 = torch.stack((a1,a2), dim=0).mean(dim=0)
m3

tensor([[-0.0090,  0.2320,  0.1711,  0.4154],
        [-0.0083,  0.1614,  0.0338,  0.5013],
        [-0.0206,  0.1466,  0.0585,  0.6754]], grad_fn=<MeanBackward1>)

In [22]:
b1 = torch.Tensor([0.7736, -0.0804,  0.0381,  0.0499])

b2 = torch.Tensor([0.1595,  0.0741,  0.0667,  0.1454])

In [25]:
c = torch.cat((b1,b2))

torch.cat((c,b1))

tensor([ 0.7736, -0.0804,  0.0381,  0.0499,  0.1595,  0.0741,  0.0667,  0.1454,
         0.7736, -0.0804,  0.0381,  0.0499])

In [ ]:
import torch.nn.functional as F

cosine_sim = F.cosine_similarity(a1,a2)

cosine_sim.shape

cosine_sim = cosine_sim.unsqueeze(-1)

cosine_sim.shape

f = torch.cat((m1,cosine_sim),dim=1)

f.shape

f



In [ ]:
classifier = nn.Linear(9,1)

labels = classifier(f)

labels

labels = labels.squeeze()

labels 
labels.shape

In [ ]:

true = torch.Tensor([1,0,1])

loss = nn.BCEWithLogitsLoss()

l = loss(labels.squeeze(),true)




In [ ]:
prova = [0,1,0]

prova = torch.Tensor(prova)

prova.long()

In [ ]:
classes = (labels>0 ).float()

classes

In [ ]:
def ciao(n :int):

    def a():
        print(n)

    def b():
        print(n+1)
    
    if n%2 == 0:
        a()
    else :
        b()
    

ciao(5)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

#compute accuracy and f1-score 
def acc_and_f1(y_pred: torch.LongTensor, y_true: torch.LongTensor):
    """
        Compute accuracy and f1-score for an epoch 
    """
    # correct = y_pred.eq(y_true)          
    # acc = correct.sum()/y_true.shape[0] 

    acc = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true,y_pred,average='macro')

    return acc,f1

#construct y_true and y_pred lists to be passed to acc_and_f1 function, but based on majority voting strategy
def majority_voting(y_pred,y_true,y_ids):
    """
        Input: the list of predicted labels, the list of corresponing true labels, the list of claim ids to compute majority voting 
        Output : the list of predicted labels via majority voting, the list of true labels (one for each claim id)
    """
    for id in y_ids:
        mask = y_ids[y_ids == id ]
        

    return 

def acc_and_f1_majority(y_pred,y_true,y_ids):

    y_pred,y_true = majority_voting(y_pred,y_true,y_ids)

    acc, f1 = acc_and_f1(y_pred, y_true)

    return acc, f1

In [50]:
import numpy as np

In [84]:
pred = torch.LongTensor([0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0])
true = torch.LongTensor([1,0,1,0,0,1,1,1,1,0,1,0,0,1,1,1,1,0,1,0,0,1,1,1])
ids  = torch.LongTensor([3,2,3,1,2,7,3,7,3,2,3,1,2,7,3,7,3,2,3,1,2,7,3,7])

y_true = []
y_pred = []
ids.unique()

for id in ids.unique():
    mask = torch.isin(ids,id)
    y_true.append(true[mask][0].item())
    y_pred.append(torch.argmax(torch.bincount(pred[mask])).item())

y_true
y_pred

tensor([1, 2, 3, 7])

[0, 0, 1, 1]

[0, 1, 0, 0]